In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_yt = pd.read_csv("youtube_recommendation_dataset -.csv")
df_yt.head()

,Title,channel_title,published_at,category_id,view_count,like_count,comment_count,favorite_count,duration,definition,caption,engagement_rate,likes_to_views_ratio,comments_to_views_ratio,duration_seconds,video_age_days
0,LA PERVERSA X LA INSUPERABLE X ALOFOKE MUSIC X...,AlofokeMusicSounds,2025-11-16 15:34:55+00:00,10,1405647,140463,9063,0,PT1M51S,hd,False,0.106375,0.099928,0.006448,111,1
1,Moana | Official Teaser,Disney,2025-11-17 17:00:47+00:00,24,2776847,26801,6684,0,PT1M,hd,True,0.012059,0.009652,0.002407,60,0
2,$0 - $1 Trillion Only FISHING in Steal a Brain...,CaylusBlox,2025-11-17 22:57:14+00:00,20,1189857,16174,1827,0,PT18M2S,hd,False,0.015129,0.013593,0.001535,1082,0
3,ALLDAY PROJECT - ‘ONE MORE TIME’ M/V,THEBLACKLABEL,2025-11-17 09:00:07+00:00,10,5319161,0,12869,0,PT3M23S,hd,True,0.002419,0.000000,0.002419,203,1
4,La Lupa | Vendetta Hero Trailer | Overwatch 2,PlayOverwatch,2025-11-17 17:00:06+00:00,20,597542,41742,4728,0,PT3M35S,hd,True,0.077768,0.069856,0.007912,215,0


In [8]:
df_yt.shape

(537, 16)

In [12]:
df_yt.isnull().sum()

Title                      0
channel_title              0
published_at               0
category_id                0
view_count                 0
like_count                 0
comment_count              0
favorite_count             0
duration                   0
definition                 0
caption                    0
engagement_rate            0
likes_to_views_ratio       0
comments_to_views_ratio    0
duration_seconds           0
video_age_days             0
dtype: int64

In [13]:
df_yt.duplicated().sum()

np.int64(0)

In [3]:
df_yt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Title                    537 non-null    object 
 1   channel_title            537 non-null    object 
 2   published_at             537 non-null    object 
 3   category_id              537 non-null    int64  
 4   view_count               537 non-null    int64  
 5   like_count               537 non-null    int64  
 6   comment_count            537 non-null    int64  
 7   favorite_count           537 non-null    int64  
 8   duration                 537 non-null    object 
 9   definition               537 non-null    object 
 10  caption                  537 non-null    bool   
 11  engagement_rate          537 non-null    float64
 12  likes_to_views_ratio     537 non-null    float64
 13  comments_to_views_ratio  537 non-null    float64
 14  duration_seconds         5

In [17]:
df_yt['Title'] = df_yt['Title'].astype('string')
df_yt['channel_title'] = df_yt['channel_title'].astype('string')
df_yt['published_at'] = pd.to_datetime(df_yt['published_at'])
df_yt['duration'] = pd.to_timedelta(df_yt['duration'])
df_yt['duration_seconds'] = pd.to_timedelta(df_yt['duration']).dt.total_seconds()

In [18]:
df_yt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   Title                    537 non-null    string             
 1   channel_title            537 non-null    string             
 2   published_at             537 non-null    datetime64[ns, UTC]
 3   category_id              537 non-null    int64              
 4   view_count               537 non-null    int64              
 5   like_count               537 non-null    int64              
 6   comment_count            537 non-null    int64              
 7   favorite_count           537 non-null    int64              
 8   duration                 537 non-null    timedelta64[ns]    
 9   definition               537 non-null    object             
 10  caption                  537 non-null    bool               
 11  engagement_rate          537 non

In [19]:
df_yt[(df_yt['like_count'] < 0) | (df_yt['comment_count'] < 0) | (df_yt['view_count'] < 0)]

,Title,channel_title,published_at,category_id,view_count,like_count,comment_count,favorite_count,duration,definition,caption,engagement_rate,likes_to_views_ratio,comments_to_views_ratio,duration_seconds,video_age_days


In [20]:
df_yt.head()

,Title,channel_title,published_at,category_id,view_count,like_count,comment_count,favorite_count,duration,definition,caption,engagement_rate,likes_to_views_ratio,comments_to_views_ratio,duration_seconds,video_age_days
0,LA PERVERSA X LA INSUPERABLE X ALOFOKE MUSIC X...,AlofokeMusicSounds,2025-11-16 15:34:55+00:00,10,1405647,140463,9063,0,0 days 00:01:51,hd,False,0.106375,0.099928,0.006448,111.0,1
1,Moana | Official Teaser,Disney,2025-11-17 17:00:47+00:00,24,2776847,26801,6684,0,0 days 00:01:00,hd,True,0.012059,0.009652,0.002407,60.0,0
2,$0 - $1 Trillion Only FISHING in Steal a Brain...,CaylusBlox,2025-11-17 22:57:14+00:00,20,1189857,16174,1827,0,0 days 00:18:02,hd,False,0.015129,0.013593,0.001535,1082.0,0
3,ALLDAY PROJECT - ‘ONE MORE TIME’ M/V,THEBLACKLABEL,2025-11-17 09:00:07+00:00,10,5319161,0,12869,0,0 days 00:03:23,hd,True,0.002419,0.000000,0.002419,203.0,1
4,La Lupa | Vendetta Hero Trailer | Overwatch 2,PlayOverwatch,2025-11-17 17:00:06+00:00,20,597542,41742,4728,0,0 days 00:03:35,hd,True,0.077768,0.069856,0.007912,215.0,0


In [21]:
threshold = df_yt['view_count'].quantile(0.75)
df_yt['is_trending'] = df_yt['view_count'] > threshold

In [23]:
df_yt['upload_hour'] = df_yt['published_at'].dt.hour
df_yt['upload_day'] = df_yt['published_at'].dt.day_name()

In [24]:
df_yt['interaction_per_day'] = df_yt['view_count'] / df_yt['video_age_days'].replace(0, 1)

In [25]:
df_yt.head()

,Title,channel_title,published_at,category_id,view_count,like_count,comment_count,favorite_count,duration,definition,caption,engagement_rate,likes_to_views_ratio,comments_to_views_ratio,duration_seconds,video_age_days,is_trending,upload_hour,upload_day,interaction_per_day
0,LA PERVERSA X LA INSUPERABLE X ALOFOKE MUSIC X...,AlofokeMusicSounds,2025-11-16 15:34:55+00:00,10,1405647,140463,9063,0,0 days 00:01:51,hd,False,0.106375,0.099928,0.006448,111.0,1,False,15,Sunday,1405647.0
1,Moana | Official Teaser,Disney,2025-11-17 17:00:47+00:00,24,2776847,26801,6684,0,0 days 00:01:00,hd,True,0.012059,0.009652,0.002407,60.0,0,False,17,Monday,2776847.0
2,$0 - $1 Trillion Only FISHING in Steal a Brain...,CaylusBlox,2025-11-17 22:57:14+00:00,20,1189857,16174,1827,0,0 days 00:18:02,hd,False,0.015129,0.013593,0.001535,1082.0,0,False,22,Monday,1189857.0
3,ALLDAY PROJECT - ‘ONE MORE TIME’ M/V,THEBLACKLABEL,2025-11-17 09:00:07+00:00,10,5319161,0,12869,0,0 days 00:03:23,hd,True,0.002419,0.000000,0.002419,203.0,1,False,9,Monday,5319161.0
4,La Lupa | Vendetta Hero Trailer | Overwatch 2,PlayOverwatch,2025-11-17 17:00:06+00:00,20,597542,41742,4728,0,0 days 00:03:35,hd,True,0.077768,0.069856,0.007912,215.0,0,False,17,Monday,597542.0
